In [1]:
#importing required libraries
import os

In [2]:
%pwd

'c:\\Trenches\\Text-Summarizer\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Trenches\\Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from txtSummarizer.constants import *
from txtSummarizer.utils.common import read_yaml, create_directory

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directory([self.config.artifacts_root])

    def get_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directory([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )

        return data_ingestion_config

In [11]:
import os
import urllib.request as request
import zipfile
from txtSummarizer.logging import logger
from txtSummarizer.utils.common import get_size

In [17]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )

            logger.info(f"Downloaded file: {filename} with the following info: {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [18]:
#creating the pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f"Exception: {e}")
    raise e

[2023-08-17 15:55:28,716: INFO: common: Config: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/ladiyusuph/Text-Summarizer/raw/main/samsum_data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}]
[2023-08-17 15:55:28,805: INFO: common: Config: {'key': 'value'}]
[2023-08-17 15:55:28,853: INFO: common: Directory already exists: artifacts]
[2023-08-17 15:55:28,906: INFO: common: Directory already exists: artifacts/data_ingestion]
[2023-08-17 15:55:28,950: INFO: 1878707940: File already exists of size: 7718.354 KB]
